# Downloading Forecast Data from ICON via Python API
This notebook guides you through the download process of a forecast file from the ICON numerical weather model via the Python API `ogd_api`. The data is provided by MeteoSwiss as part of Switzerland’s Open Government Data (OGD) initiative.

---

## 🔍 **What You’ll Do in This Notebook**

1. 🛰️  **Retrieve**  
    Download deterministic forecast data (e.g., soil temperature from ICON) via the `ogd_api` module.

2. 🧭  **Explore**  
    Use ecCodes to explore the forcast data file.
---

## 1. Retrieving Forecast Data
In this first part, we retrieve soil temperature forecast data from the ICON numerical weather model. To download this data, we use the `ogd_api` module from the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — a convenient interface for accessing numerical weather forecasts via the [STAC (SpatioTemporal Asset Catalog) API](https://data.geo.admin.ch/api/stac/static/spec/v1/apitransactional.html#tag/Data/operation/getAsset), which provides structured access to Switzerland’s open geospatial data.

#### 📁  Browsing the STAC Catalog (Optional)

If you'd like to explore the ICON forecast datasets interactively before downloading, you can browse them directly in the STAC catalog:

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH1-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en)

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH2-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch2?.language=en)


Below is a screenshot of the ICON-CH2-EPS collection as seen in the STAC browser interface.

![browser-ch2-resized.PNG](./images/browser-ch2-resized.PNG)

⚠️ **Temporary Setup Notice**

We currently use **ecCodes 2.36.0**, which requires manually setting the `ECCODES_DIR` environment variable to locate the binary library.
The [install_eccodes.sh](./install_eccodes.sh) script prints the full path to the installed environment — use that path here.

From **ecCodes 2.37.0** onwards, this step won't be needed as the Python wheel will include the binary automatically.

In [1]:
from pathlib import Path
import os

# Replace this with the actual environment path if different
os.environ["ECCODES_DIR"] = str(Path.home()) + "/miniconda3/envs/eccodes_env"

### Creating a Request
To retrieve the forecast data, we first define an API request using the `ogd_api.Request` class.
In this example the request targets lead time zero, which is the forecast's start time also called reference or initialization datetime.

>⏰ **Forecast Availability**: Forecast data will typically be available a couple of hours after the reference time — due to the model runtime and subsequent upload time. The data remains accessible for 24 hours after upload.

In [2]:
from meteodatalab import ogd_api


req = ogd_api.Request(
    collection="ogd-forecasting-icon-ch1",
    variable="T_SO",
    reference_datetime="2025-04-13T12:00:00Z", # make sure to change the date
    perturbed=False,
    horizon="P0DT0H",
)

Each argument in the request serves the following purpose:

| Argument             | Description |
|----------------------|-------------|
| `collection`         | Forecast collection to use (e.g., `ogd-forecasting-icon-ch1`). |
| `variable`           | Meteorological variable of interest (`T_SO` = soil temperature). |
| `reference_datetime` | Initialization time of the forecast in **UTC** (e.g., `2025-04-13T12:00:00Z`). |
| `perturbed`          | If `True`, retrieves ensemble forecast members; if `False`, returns the deterministic forecast. |
| `horizon`            | Forecast lead time, provided as either:<br>– `ISO 8601` duration string (e.g., `P0DT0H`)<br>– `datetime.timedelta` object (e.g., `datetime.timedelta(hours=0)`) |

### Downloading Data
We now send our request to the API and download the resulting dataset using the `download_from_ogd()` function. The function expects a request and a path to a directory, where the data will be stored. In this example the forecast file is stored in the folder "forecast_files" within this directory.

> 💡 **Tip**: Use temporary caching with earthkit-data to skip repeated downloads — it's auto-cleaned after the session.
> *For more details, see the [earthkit-data caching docs](https://earthkit-data.readthedocs.io/en/latest/examples/cache.html)*.

> 💡 **Hint**: If you get an error message containing "HTTPError: 403 Client Error: Forbidden for url", you may be trying to retrieve data older than 24h hours! Please adjust your requests.

In [3]:
from earthkit.data import config
config.set("cache-policy", "temporary")

ogd_api.download_from_ogd(req, Path.cwd().joinpath("forcast_files"))

After executing the code block above, check that the download has been completed. In the folder "forecast_files" you should see three GRIB files and three SHA256 files:
- horizontal_constants_icon-ch1-eps.grib2
- horizontal_constants_icon-ch1-eps.sha256
- icon-ch1-eps-202504131200-0-t_so-ctrl.grib2
- icon-ch1-eps-202504131200-0-t_so-ctrl.sha256
- vertical_constants_icon-ch1-eps.grib2
- vertical_constants_icon-ch1-eps.sha256

## 2. How to read GRIB files